<div class="alert alert-block alert-info">
    <img src='melia.png' width="200" height="200"/>
    <h1><center>Melia Paradius Cancún</center></h1>  
    <p><center>Set competitivo de FACC</center></p>  
</div>

<center>Importaciones</center>

In [11]:
###
# Importadción de librerias del sistema
###
import os
import time
import datetime
from datetime import date
import random
import csv
print("Librerias del sistema listas")
###
# Importación de librerias para selenium
###
try:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    print("Librerias para selenium listas")
except():
    print("Instalando selenium en el enviroment")
    import sys
    !{sys.executable} -m pip install -U selenium
###
# Extracción del precio del dolar hoy
###
try:
    from forex_python.converter import CurrencyRates
except():
    print("Instalando la libreria CurrencyrRates")
    import sys
    !{sys.executable} -m pip install -U forex-python
###
# Importación de librerias para dataframe
###
import pandas as pd
print("Librerias para pandas listas")

Librerias del sistema listas
Librerias para selenium listas
Librerias para pandas listas


<center>Precio dolar al día</center>

In [12]:
c = CurrencyRates()
mxn = c.get_rate('USD', 'MXN')
mxn

24.2688102894

<center>Métodos</center>

In [13]:
###
# Métodos para MELIA CANCÚN
###
def melia_connect():
    """Este método ayuda a poder realizar la conexión a la página de Melia Cancún"""
    url = "https://www.melia.com/en/hotels/mexico/cancun/paradisus-cancun/index.htm?esl-k=sem-google%7Cng%7Cc319560739894%7Cme%7Ckmelia%20cancun%7Cp%7Ct%7Cdc%7Ca67489157927%7Cg1658904174&&gclid=Cj0KCQiAiZPvBRDZARIsAORkq7ezSAW0BaUwbafjCDaAVVIWnysjsq89sphGT01Fn7FTveTMLkA-xT8aAtt6EALw_wcB&gclsrc=aw.ds"
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--incognito")
    #prefs = {"download.default_directory" : os.getcwd()+"/Resultados Selenium"} 
    #chromeOptions.add_experimental_option("prefs",prefs)
    #Agregando un proxy
    #proxy = str(random.randrange(1, 255))+'.'+str(random.randrange(1,255))+'.'+str(random.randrange(1,255))+'.'+str(random.randrange(1,255))+':'+str(random.randrange(8000,65535))
    #chromeOptions = melia_proxy(chromeOptions)
    driver = webdriver.Chrome("chromedriver.exe",options=chromeOptions)
    driver.get(url) 
    melia_wait(3)
    return driver

def melia_proxy(chromeOptions,proxy):
    """Crea una nueva proxy, para poder ocultar tu IP"""
    return chromeOptions.add_argument('--proxy-server=proxy')

def melia_wait(num):
    """Ayuda a la espera humana, para evitar el vaneo"""
    time.sleep(num)

def melia_first_form(driver,date1,date2):
    """Llena el primer formulario"""
    #print("Entre a la funcion")
    #print(driver,date1,date2)
    melia_wait(2)
    string_desplegabledates = 'div[id*="mbe-dates-select"]' 
    desplegable_Dates = driver.find_element_by_css_selector(string_desplegabledates)
    #print("Fechas desplegadas correctamente.")
    driver.execute_script("arguments[0].click();", desplegable_Dates)
    #click_dates = desplegable_Dates.click()
    melia_wait(2)
    try:
        string_date1 = 'li[d='+'"'+date1+'"'+']'
        date_1 = driver.find_element_by_css_selector(string_date1)
        #print("Fecha 1 seleccionada")
        driver.execute_script("arguments[0].click();", date_1)
        melia_wait(1)
    except:
        print("The date: "+date_1+" is not aviable.")
    try:
        string_date2 = 'li[d='+'"'+date2+'"'+']'
        date_2 = driver.find_element_by_css_selector(string_date2)
        #print("Fecha 2 seleccionada")
        driver.execute_script("arguments[0].click();", date_2)
        melia_wait(1)
    except:
        print("The date: "+date_2+" is not aviable.")
    melia_wait(3)
    string_button = "button[id='mbe-search-button']"
    search = driver.find_element_by_css_selector(string_button)
    #print("Se oprimió el botón búscar")
    driver.execute_script("arguments[0].click();", search)
    melia_wait(1)
    
def choose_rooms(driver,item):
    string_prices = "a[class*='btn btn-primary'][data-nombre-habitacion='"+item+"']"
    flag_dolar = True
    try:
        prices = driver.find_element_by_css_selector(string_prices)
        driver.execute_script("arguments[0].click();", prices)
        price = driver.find_element_by_css_selector("div[class*='js-totalAmountWithTax-currency currency-left']")
        precio = price.get_attribute("data-precio")
        moneda = price.get_attribute("data-nueva-moneda")
        if "," in precio:
            precio = precio.replace(",","")
        if "MX" in moneda:
            flag_dolar = False
        back_page(driver)
        melia_wait(2)
    except:
        print("La habitación: "+item+" no aparece")
        precio = 0
    print(precio)
    return precio,flag_dolar

def back_page(driver):
    driver.execute_script("window.history.go(-1)")
    
def back_initial(driver):
    driver.execute_script("window.location.href = 'https://www.melia.com/en/hotels/mexico/cancun/paradisus-cancun/index.htm?esl-k=sem-google%7Cng%7Cc319560739894%7Cme%7Ckmelia%20cancun%7Cp%7Ct%7Cdc%7Ca67489157927%7Cg1658904174&&gclid=Cj0KCQiAiZPvBRDZARIsAORkq7ezSAW0BaUwbafjCDaAVVIWnysjsq89sphGT01Fn7FTveTMLkA-xT8aAtt6EALw_wcB&gclsrc=aw.ds'")

<center>Operación</center>

In [14]:
dicc_melia = {"Hotel":[],
             "Fecha_consulta":[],
             "Fecha_reserva_checkin":[],
             "Fecha_reserva_checkout":[],
             "Nombre_habitacion":[],
             "Precio_habitacion_MXN":[],
             "Precio_habitacion_USD":[],
             "Precio_dolar_al_dia":[]}
melia_data = pd.DataFrame(dicc_melia)
###
# Variables globales
###
rooms_list = ['PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS',
              'PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI',
              'LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE']
consult_Date = date.today()
today = datetime.date.today()
tomorrow = today + datetime.timedelta(days=1)
number_of_days = int(input("Ingresa el número de dias a iterar a partir de hoy "))
###
# Conexión, manejar excepciones, evitará baneo de las ips estáticas.
###
driver = melia_connect()
while number_of_days != 0:
        print(number_of_days)
        print(today," ",tomorrow)
        date1 = str(today.year)+"/"+str(today.month)+"/"+str(today.day)
        date2 = str(tomorrow.year)+"/"+str(tomorrow.month)+"/"+str(tomorrow.day)
        melia_wait(3)
        melia_first_form(driver,date1,date2)
        for item in rooms_list:
            precio,flag = choose_rooms(driver,item)
            if flag == True:
                dicc_data = {"Hotel":"Melia Paradisus Cancún",
                     "Fecha_consulta":str(consult_Date),
                     "Fecha_reserva_checkin":str(today),
                     "Fecha_reserva_checkout":str(tomorrow),
                     "Nombre_habitacion":item,
                     "Precio_habitacion_MXN":float(precio)*mxn,
                     "Precio_habitacion_USD":float(precio),
                     "Precio_dolar_al_dia":mxn}
                melia_data = melia_data.append(dicc_data,ignore_index = True)
            else:
                dicc_data = {"Hotel":"Melia Paradisus Cancún",
                     "Fecha_consulta":str(consult_Date),
                     "Fecha_reserva_checkin":str(today),
                     "Fecha_reserva_checkout":str(tomorrow),
                     "Nombre_habitacion":item,
                     "Precio_habitacion_MXN":float(precio),
                     "Precio_habitacion_USD":float(precio)/mxn,
                     "Precio_dolar_al_dia":mxn}
                melia_data = melia_data.append(dicc_data,ignore_index = True)
        melia_wait(3)
        back_initial(driver)
        melia_wait(3)
        today = tomorrow
        tomorrow = today + datetime.timedelta(days=1)
        number_of_days -= 1
driver.quit()

Ingresa el número de dias a iterar a partir de hoy 90
90
2020-04-08   2020-04-09
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE no aparece
0
89
2020-04-09   2020-04-10
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE no aparece
0
88
2020-04-10   2020-04-11
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE no aparece
0
87
2020-04-11   2020-04-12
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIO

57
2020-05-11   2020-05-12
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE no aparece
0
56
2020-05-12   2020-05-13
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE no aparece
0
55
2020-05-13   2020-05-14
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE no aparece
0
54
2020-05-14   2020-05-15
La habitación: PARADISUS JUNIOR SUITE LAGOON VIEW ALL INCLUS no aparece
0
La habitación: PARADISUS JUNIOR SUITE OCEAN VIEW ALL INCLUSI no aparece
0
La habitación: LUXURY JUNIOR SUITE OCEAN VIEW ALL INCLUSIVE no aparece
0
53
2020-

In [15]:
melia_data.to_csv("./Melia_data/Melia_"+str(consult_Date)+"_comma.csv",quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")
melia_data.to_csv("./Melia_data/Melia_"+str(consult_Date)+"_pipe.csv",sep='|',quoting=csv.QUOTE_ALL,index=False,encoding="utf-8-sig")